In [3]:
# Import libs
from dataclasses import dataclass
import pandas as pd
from pandas.core.base import DataError
import requests
import chess.pgn
import chess

In [4]:
def get_data(username,timestamp):
    # Get the data from the API
    print("Getting data from API...")
    url = f"http://lichess.org/api/games/user/{username}?since={timestamp}"
    data = requests.get(url)
    if data.status_code != 200:
        raise Exception("The following response was returned: " + str(data.status_code))
    else:
        print("Successfully retrieved data")
        # Write data to file
        with open("data.pgn", "w") as f:
            f.write(data.text)

def pgn_to_df(pgn_file):
    totalkeyerrors = 0
    print("Converting pgn file to dataframe...")
    pgn = open(pgn_file)
    game = chess.pgn.read_game(pgn)

    print(game)
    df = pd.DataFrame(columns=["Event","Site","Date","White","Black","Result","UTCDate","UTCTime","WhiteElo","BlackElo","WhiteRatingDiff","BlackRatingDiff","Variant","TimeControl","ECO","Termination","FEN","SetUp"])
    f = open("data.pgn", "r")
    game_list = []
    while True:
        game = chess.pgn.read_game(f)
        if game is None:
            break
        game_list.append(game)

    for game in game_list:
        try:
            df = df.append({"Event":game.headers["Event"],"Site":game.headers["Site"],"Date":game.headers["Date"],"White":game.headers["White"],"Black":game.headers["Black"],"Result":game.headers["Result"],"UTCDate":game.headers["UTCDate"],"UTCTime":game.headers["UTCTime"],"WhiteElo":game.headers["WhiteElo"],"BlackElo":game.headers["BlackElo"],"WhiteRatingDiff":game.headers["WhiteRatingDiff"],"BlackRatingDiff":game.headers["BlackRatingDiff"],"Variant":game.headers["Variant"],"TimeControl":game.headers["TimeControl"],"ECO":game.headers["ECO"],"Termination":game.headers["Termination"],"FEN":game.headers["FEN"],"SetUp":game.headers["SetUp"]},ignore_index=True)

        except KeyError:
            #print(KeyError)
            totalkeyerrors += 1
        except DataError:
            print("DataError")
        game = chess.pgn.read_game(pgn)
    print("Total keyErrors: " + str(totalkeyerrors))
    return df

In [6]:
# Get data from the player second argument is a unix timestamp
#get_data("lonaya","1639554963")
# Look at our dataframe
df1 = pgn_to_df("data.pgn")
print(df1)

Converting pgn file to dataframe...
[Event "Rated Blitz game"]
[Site "https://lichess.org/FPjEmjjr"]
[Date "2021.12.20"]
[Round "?"]
[White "anand68"]
[Black "lonaya"]
[Result "1-0"]
[BlackElo "2669"]
[BlackRatingDiff "-4"]
[ECO "A53"]
[Termination "Normal"]
[TimeControl "180+2"]
[UTCDate "2021.12.20"]
[UTCTime "09:09:39"]
[Variant "Standard"]
[WhiteElo "2778"]
[WhiteRatingDiff "+4"]

1. d4 Nf6 2. c4 d6 3. g3 g6 4. Bg2 Bg7 5. Nf3 O-O 6. O-O Re8 7. Nc3 c5 8. d5 Nbd7 9. Bf4 Rb8 10. a4 a6 11. h3 Nf8 12. e4 h6 13. Qd2 Kh7 14. Rfe1 g5 15. Be3 N6d7 16. Qc2 Ng6 17. Nd2 Nde5 18. f4 gxf4 19. gxf4 Nh4 20. fxe5 Rg8 21. Nf3 Nxg2 22. Kxg2 Bxe5+ 23. Kh1 Rg3 24. Nxe5 Rxh3+ 25. Kg1 Qg8+ 26. Kf1 dxe5 27. Bf2 Qg5 28. Ne2 Rh2 29. Ng3 Bh3+ 30. Ke2 Qxg3 31. Rf1 Qg4+ 32. Ke1 Qf4 33. Rg1 Bg4 34. Ra3 Rg8 35. Rag3 h5 36. R3g2 Rg6 37. Rxh2 Qxh2 38. Qd3 Qf4 39. Qe3 Qh2 40. Qxc5 Rf6 41. Qe3 Kg6 42. Qg3 Qxg3 43. Bxg3 Rf3 44. Kd2 Rb3 45. Kc2 Re3 46. Bxe5 Rxe4 47. Bc3 Rxc4 48. Re1 e6 49. d6 Rc8 50. Kb3 e5 51. Bxe5 R

In [ ]:
# Visualize Stuff